<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Natural Language Processing  - Neural Network - SPAM or HAM
</b></font> </br></p>

---

# 0  | Install & Import
***

In [ ]:
# Install

In [ ]:
# Import
import numpy as np
from pandas import read_csv, DataFrame, concat

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)

from keras.utils import set_random_seed, plot_model

from tensorflow import keras
from tensorflow.config.experimental import enable_op_determinism
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import plotly.express as px
import plotly.subplots as sp

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1 | Understand
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Aufgabe verstehen</br>
✅ Daten sammeln</br>
✅ Statistische Analyse (Min, Max, Mean, Korrelation, ...)</br>
✅ Datenvisualisierung (Streudiagramm, Box-Plot, ...)</br>
✅ Prepare Schritte festlegen</br>

<p><font color='black' size="5">
Anwendungsfall
</font></p>

---


Diese DataSet wurde aus Recherchequellen im Internet zusammengestellt: -> Eine Sammlung von 425 SMS-Spam-Nachrichten wurde manuell von der Grumbletext-Website extrahiert. Dies ist ein britisches Forum, in dem Mobiltelefonnutzer öffentliche Behauptungen über SMS-Spam-Nachrichten aufstellen, die meisten von ihnen, ohne die empfangene Spam-Nachricht selbst zu melden. Die Identifizierung des Textes von Spam-Nachrichten in den Ansprüchen ist eine sehr schwierige und zeitaufwändige Aufgabe und erforderte das sorgfältige Scannen von Hunderten von Webseiten.

Das Ziel ist es, ein Vorhersagemodell zu erstellen, das bestimmt, ob eine Textnachricht Spam oder Ham ist.


[Info](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)

[DataSet](https://archive-beta.ics.uci.edu/dataset/228/sms+spam+collection)






Foto von <a href="https://unsplash.com/@nhillier?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Nick Hillier</a> auf <a href="https://unsplash.com/de/fotos/yD5rv8_WzxA?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>


In [ ]:
df = read_csv(
    "https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/smsspamcollection.csv",
    sep="\t",
    header=None,
    names=["class", "msg"],
)
data = df["msg"].copy()
target = df["class"].replace(["ham", "spam"], [0, 1]).copy()

In [ ]:
data.shape, target.shape

In [ ]:
data[26]  # Lol, du bist immer so überzeugend (lol = Laughing out loud)

# 2 | Prepare
---


<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Datentyp ermitteln/ändern</br>
✅ Train-Test-Split durchführen</br>
✅ Nicht benötigte Features löschen</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>

<p><font color='black' size="5">
Tokenizer
</font></p>

Der Datensatz wird tokenisiert (`Tokenizer`). Jedem Wort wird eine eindeutige Nummer zugewiesen - die für das neuronale Netzwerk erforderlich ist, um die Eingabe zu interpretieren.

[Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer)

In [ ]:
num_words_ = 10000  # die maximale Anzahl der zu behaltenden Wörter, basierend auf der Worthäufigkeit
oov_tok_ = "<OOV>"  # wird zu word_index hinzugefügt & und bei fehlenden Wörte bei text_to_sequence-Aufrufen verwendet

In [ ]:
tokenizer = Tokenizer(num_words=num_words_, oov_token=oov_tok_)
tokenizer.fit_on_texts(data)
word_index = tokenizer.word_index

In [ ]:
type(word_index), len(word_index)

In [ ]:
[(key, value) for key, value in word_index.items()][3019:3024]

<p><font color='black' size="5">
Text 2 Sequenz
</font></p>

Diese Tokens (Wörter) werden dann in Sequenzen sortiert, um sicherzustellen, dass die Tokens für jedes Wort der richtigen Reihenfolge folgen, wie sie von jedem Satz vorgegeben wird.


[text_to_word_sequence](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/text_to_word_sequence)

In [ ]:
sequences = tokenizer.texts_to_sequences(data)

In [ ]:
type(sequences), len(sequences)

In [ ]:
sequences[26]

<p><font color='black' size="5">
Padding
</font></p>

Damit jeder Satz gleich lang ist, werden am Ende eines Satzes Nullen einfügt. Dies ist erforderlich, wenn ein Satz länger als ein anderer ist, da jeder Satz zum Zweck der Analyse durch das RNN gleich lang sein muss.

[pad_sequence](https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences)

In [ ]:
max_length = 100  # optional, maximale Länge der Sequenzen
trunc_type = "post"  # Entfernt Worte aus Sequenzen, die größer als maxlen sind, entweder am Anfang oder am Ende der Sequenzen
padding_type = "post"  # Auffüllen entweder vor oder nach jeder Sequenz

In [ ]:
data_padded = pad_sequences(
    sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type
)

In [ ]:
data_padded[26]

<p><font color='black' size="5">
Train-Test-Split
</font></p>

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data_padded, target, test_size=0.2, random_state=42, stratify=target
)
data_train.shape, data_test.shape, target_train.shape, target_test.shape

# 3 | Modeling
---

<p><font color='black' size="5">
Modellierung eines Neuronalen Netzes
</font></p>

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellauswahl treffen</br>
✅ Pipeline erweitern/konfigurieren</br>
✅ Training durchführen</br>
✅ Hyperparameter Tuning</br>
✅ Cross-Valdiation</br>
✅ Bootstrapping</br>
✅ Regularization</br>

[Core layers](https://keras.io/api/layers/core_layers/)
[Reshaping layers](https://keras.io/api/layers/reshaping_layers/)
[Layer activation functions](https://keras.io/api/layers/activations/)


<p><font color='black' size="5">
Zufallszahl initialisieren
</font></p>

In [ ]:
set_random_seed(42)
enable_op_determinism()

<p><font color='black' size="5">
Modellaufbau
</font></p>

In [ ]:
input_dim_ = num_words_
output_dim_ = 16  # semantische "Tiefe" - Dimensionen Embeddings

In [ ]:
model = Sequential()
model.add(Embedding(input_dim_, output_dim_, input_length=max_length))
model.add(Flatten())
model.add(Dense(6, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

<p><font color='black' size="5">
Einfaches Layer-Diagramm
</font></p>

In [ ]:
model.summary()

In [ ]:
# Visualisierung neuronales Netz
model.build(input_shape=(None, max_length)) # Build the model
plot_model(
    model,
    to_file="nn_structure.png",
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    dpi=100,
    expand_nested=True,
    show_layer_activations=True
)

In [ ]:
# Anzahl Parameter je Layer
for layer in model.layers:
    print(f"{layer.name}: {layer.count_params()} Parameter")

<p><font color='black' size="5">
Training
</font></p>


In [ ]:
epochs_ = 10
model.fit(data_train, target_train, validation_split=0.2, epochs=epochs_)

In [ ]:
print(model.history.params)
print(model.history.history.keys())

<p><font color='black' size="5">
Loss-Entwickung
</font></p>

In [ ]:
title_ = "Loss-Entwicklung"
px.line(
    y=model.history.history["loss"],
    title=title_,
    labels={"x": "Epochen", "y": "Loss-Wert"},
    width=800,
    height=400,
)

# 4 | Evaluate
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Prognose (Train, Test) erstellen</br>
✅ Modellgüte prüfen</br>
✅ Residuenanalyse erstellen</br>
✅ Feature Importance/Selektion prüfen</br>
✅ Robustheitstest erstellen</br>
✅ Modellinterpretation erstellen</br>
✅ Sensitivitätsanalyse erstellen</br>
✅ Kommunikation (Key Takeaways)</br>

<p><font color='black' size="5">
Prognose
</font></p>

In [ ]:
target_train_pred = model.predict(data_train)
target_test_pred = model.predict(data_test)

<p><font color='black' size="5">
Confusion Matrix
</font></p>

In [ ]:
target_train_pred_ = np.around(target_train_pred, 0)
target_test_pred_ = np.around(target_test_pred, 0)

In [ ]:
conf_matrix = confusion_matrix(target_test, target_test_pred_)
display_labels_ = ["No Spam", "Spam"]
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=display_labels_)
disp.plot(cmap="Blues")

In [ ]:
print(
    classification_report(target_test, target_test_pred_, target_names=display_labels_)
)

<p><font color='black' size="5">
Accuracy
</font></p>

In [ ]:
acc_train = accuracy_score(target_train, target_train_pred_) * 100
print(f"Modell: {model} -- Train -- Accuracy: {acc_train:5.2f}")

In [ ]:
acc_test = accuracy_score(target_test, target_test_pred_) * 100
print(f"Modell: {model} -- Test -- Accuracy: {acc_test:5.2f}%")

<p><font color='black' size="5">
Real vs. Predict
</font></p>

In [ ]:
type(target_test_pred)

In [ ]:
for row in range(200, 210):
    print(
        f"Real:  {np.array(target_test)[row]} Pred: {float(target_test_pred_[row]):.2f}"
    )

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


def decode_review(text):
    return " ".join([reverse_word_index.get(i, "") for i in text])

In [ ]:
for row in range(200, 210):
    print(
        "Real: ",
        np.array(target_test)[row],
        "Pred: ",
        int(np.around(float(target_test_pred_[row]), 0)),
        " -- ",
        decode_review(data_test[row]),
        sep=" ",
    )

# 5 | Deploy
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellexport und -speicherung</br>
✅ Abhängigkeiten und Umgebung</br>
✅ Sicherheit und Datenschutz</br>
✅ In die Produktion integrieren</br>
✅ Tests und Validierung</br>
✅ Dokumentation & Wartung</br>